# X - HEC Embeddings 2 : Advanced Word Representations

In this practical session, we will focus on word embeddings through word2vec and a simple classification model for sentiment analysis. Once a word2vec skipgram is trained, we can visualize learned word vectors in a reduced space and use them in our classification model.

In [1]:
import io
import os
import re
import warnings
import itertools

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from ast import literal_eval

import tensorflow as tf
import tensorflow_datasets as tfds

from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

/anaconda3/envs/tensorflow_env/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Load Data

In [2]:
# Define data path
file_path = os.getcwd() + '/clean_full_graph.csv.gzip'

# Read csv file with right parameters
df_all = pd.read_csv(file_path, 
                     compression='gzip', 
                     low_memory=False, 
                     parse_dates=['review_date', 'review_date_diner'])

df_all.head(2)

,review_id,review_date,review_date_diner,review_has_answer,review_rating,review_rating_value,review_rating_service,review_rating_atmosphere,review_rating_food,review_title,...,rest_rating_excellent,rest_rating_very_good,rest_rating_neutral,rest_rating_poor,rest_rating_terrible,rest_url,rest_url_menu,rest_adress,rest_description,grp
0,g191301-d4453079-r728219948,2019-11-22,2019-11-01,True,4.5,NaN,NaN,NaN,NaN,Birthday Shots shots shots!,...,833,101,43.0,34.0,65.0,https://www.tripadvisor.com/ShowUserReviews-g1...,http://www.revolution-bars.co.uk/bar/london-ri...,4 Whittaker Avenue,"Perched on the Thames riverside, this beautifu...",cap
1,g191301-d4453079-r728632295,2019-11-24,2019-11-01,True,4.5,NaN,NaN,NaN,NaN,Louis!!!,...,833,101,43.0,34.0,65.0,https://www.tripadvisor.com/ShowUserReviews-g1...,http://www.revolution-bars.co.uk/bar/london-ri...,4 Whittaker Avenue,"Perched on the Thames riverside, this beautifu...",cap


For the current exercice we'll work with Capgemini Invent's dataset, so that everyone has the same data. Later on you could try to do your own embedding with your scrapped data and compare the results.

In [ ]:
df_cap = df_all[df_all.grp == 'cap'].reset_index(drop=True)
df_cap.head(2)

In [ ]:
df_cap = df_cap[df_cap['review_content'].str.len() >= 4]

# Tokenization & Text Encoding
This part concerns tokenization and text encoding with TensorFlow modules :

*(i) Build the token vocabulary* <br>
*(ii) Build a text encoder relying each word to an index, and thus each text to a sequence of word indices* (```list```) <br>
*(iii) Build a TensorFlow dataset for word2vec training*


1. **Tokenization** : Build Vocabulary

In [ ]:
DATASET_SIZE = 10000

df_cap['review_content'] = df_cap['review_content'].apply(lambda x : literal_eval(x)[0])

reviews = df_cap['review_content'][:DATASET_SIZE].values.tolist()
tokenizer = tfds.features.text.Tokenizer()
vocabulary_set = set()

for text in tqdm_notebook(reviews) :
    
    tokens = tokenizer.tokenize(text)
    vocabulary_set.update(tokens)
    
vocab_size = len(vocabulary_set)
vocab_size

2. **Token Encoding**

In [ ]:
token_encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

print(df_cap['review_content'][0])

print('\n')

token_encoded_text = token_encoder.encode(df_cap['review_content'][0])
print(token_encoded_text)

print('\n')

token_decoded_text = token_encoder.decode(token_encoded_text)
print(token_decoded_text)

In [ ]:
for tk in token_encoded_text[:10] :
    
    print('{} ----> {}'.format(tk, token_encoder.decode([tk])))

3. **Build Learning Dataset**

To learn word2vec vectors, we define center and context words. Thus, we concatenate each document, i.e. sequence of word indices to make the moving context window possible.

*Question : build this corpus of concatenated encoded documents, i.e. transform a list of* ```DATASET_SIZE``` *lists of indices in one list.*

In [ ]:
 #### FILL THE BLANK(S) ####
all_tokens = 
len(all_tokens)

The ```tf.keras.preprocessing.sequence.make_sampling_table``` and ```tf.keras.preprocessing.sequence.skipgrams``` sequence preprocessing functions are useful to build the negative sampled dataset. The ```skipgrams```function allows to browse the entire document (```all_tokens```) to identify positive and negative pairs of center and context words.

*Question : use these functions to get skipgram word pairs with labels (positive or negative). You can use the default* ```sampling_factor```, ```window_size```=4 *and generate one* ```negative_samples```.

In [ ]:
#### FILL THE BLANK(S) ####

sampling_table = 

#### FILL THE BLANK(S) ####
X, y = 

For word2vec training, we won't use a validation dataset this time. In eventual further experimentations, you could include it !

In [ ]:
BUFFER_SIZE = len(X)
BATCH_SIZE = 2048

# threshold_set = int(np.floor(0.8*len(X)))
# 
# X_train, y_train = tf.convert_to_tensor(X[:threshold_set], dtype=tf.int32), tf.convert_to_tensor(y[:threshold_set], dtype=tf.int32)
# center_words_train, context_words_train = X_train[:,0], X_train[:,1]
# 
# word2vec_dataset_train = tf.data.Dataset.from_tensor_slices((center_words_train, context_words_train, y_train)).batch(BATCH_SIZE)
# 
# X_test, y_test = tf.convert_to_tensor(X[threshold_set:], dtype=tf.int32), tf.convert_to_tensor(y[threshold_set:], dtype=tf.int32)
# center_words_test, context_words_test = X_test[:,0], X_test[:,1]
# 
# word2vec_dataset_test = tf.data.Dataset.from_tensor_slices((center_words_test, context_words_test, y_test)).batch(BATCH_SIZE)

X, y = tf.convert_to_tensor(X, dtype=tf.int32), tf.convert_to_tensor(y, dtype=tf.int32)
center_words, context_words = X[:,0], X[:,1]

word2vec_dataset = tf.data.Dataset.from_tensor_slices((center_words, context_words, y)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# Word2vec SkipGram with Negative Sampling

In [ ]:
center_words_batch, context_words_batch, y_batch = next(iter(word2vec_dataset))
print(center_words_batch.shape, context_words_batch.shape, y_batch.shape, sep='\n')

1. **SkipGram Model**

In [ ]:
class SkipGram(tf.keras.Model):
    
    def __init__(self, d_model, token_vocab_size):
        
        super(SkipGram, self).__init__()
        
        self.d_model = d_model
        self.vocab_size = token_vocab_size
        
        self.input_embedding = tf.keras.layers.Embedding(self.vocab_size, self.d_model, name='input_embedding')
        self.output_embedding = tf.keras.layers.Embedding(self.vocab_size, self.d_model, name='output_embedding')
        
        
    def call(self, center_word, context_word):
        
        center_vector = self.input_embedding(center_word)
        context_vector = self.output_embedding(context_word)
        
        dot_product = tf.math.reduce_sum(tf.multiply(center_vector, context_vector), axis=1)
        
        return tf.math.sigmoid(dot_product) # loss : from_logits=False

In [ ]:
skipgram = SkipGram(300, token_encoder.vocab_size)
assert skipgram(center_words_batch, context_words_batch).shape[0] == BATCH_SIZE

2. **Optimization Objective**

In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')

# test_loss = tf.keras.metrics.Mean(name='test_loss')
# test_accuracy = tf.keras.metrics.BinaryAccuracy(name='test_accuracy')

3. **Training Loop**

In [ ]:
def train_step(center_word, context_word, label):
    
    with tf.GradientTape() as tape :
        
        prediction = skipgram(center_word, context_word)
        loss = loss_object(label, prediction)
    
    gradient = tape.gradient(loss, skipgram.trainable_variables)
    optimizer.apply_gradients(zip(gradient, skipgram.trainable_variables))

    train_loss(loss)
    train_accuracy(label, prediction)

In [ ]:
# def test_step(center_word, context_word, label):
#     
#     prediction = skipgram(center_word, context_word)
#     loss = loss_object(label, prediction)
# 
#     test_loss(loss)
#     test_accuracy(label, prediction)

One epoch can take almost 10 minutes so define a small number of epochs to end the hands on !

In [ ]:

#### FILL THE BLANK(S) ####
EPOCHS = 

DATASET_LENGTH = X.shape[0] # threshold_set


for epoch in tqdm_notebook(iterable = range(EPOCHS), total = EPOCHS, desc = 'T R A I N I N G') :
    
    train_loss.reset_states()
    train_accuracy.reset_states()
    # test_loss.reset_states()
    # test_accuracy.reset_states()
    
    for center_word, context_word, label in tqdm_notebook(word2vec_dataset, 
                                                          total = int(np.ceil(DATASET_LENGTH/BATCH_SIZE)), 
                                                          desc = 'Epoch {}/{}'.format(epoch+1, EPOCHS)) : 
        
        train_step(center_word, context_word, label)
    
    
    # for center_word, context_word, label in word2vec_dataset_test :
    #     
    #     test_step(center_word, context_word, label)
    

    print ('Loss : {} - Accuracy : {}\n'.format(train_loss.result(), 
                                                train_accuracy.result()*100))

In [ ]:
#skipgram.save_weights("'./checkpoints/skipgram")

# Embedding Visualization

In [ ]:
skipgram.summary()

*Questions : get the weights of the* ```input_embedding``` *layer and store them in a variable* ```weights```. *These weights correspond to word2vec skipgram embeddings.*

In [ ]:
weights = #### FILL THE BLANK(S) ####

print(weights, weights.shape, sep='\n')

*Question : use the following cell to store the learned vectors in the correct format and visualize them in the Embedding Projector.*

In [ ]:
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for idx, word in enumerate(token_encoder.tokens):
    
    vec = weights[idx+1] # skip 0, it's padding.
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in vec]) + "\n")
    
out_v.close()
out_m.close()

To visualize your trained embeddings, open the [Embedding Projector](http://projector.tensorflow.org/) :

- Click on "Load data".
- Upload the two files we created above : vecs.tsv and meta.tsv.

The embeddings will now be displayed. You can search for words from ```token_encoder.tokens``` to find their closest neighbors.

# Text Classication : Sentiment Analysis
Our learned word embeddings can be used to represent the words of a text and to build a text representation. This text representation will be useful for classifcation.

*Question : in this part, our objective is to train a text classification model for sentiment analysis using the first* ```DATASET_SIZE``` reviews*. You have to :*
1. *Build the learning dataset : use the ```review_content``` column for text inputs and ```rest_rating``` for sentiments (outputs).* **NB** *: Don't forget to encode and pad your text, and use one-hot encoding for sentiment labels.*
2. *Once the learning dataset built, you have to split it into train and validation datasets.*
3. *Define and train a text classification model.*

As indications, you can use ```tf.keras.preprocessing.sequence.pad_sequences``` for text padding and ```tf.keras.utils.to_categorical``` for one-hot label encoding. For your first model, you can already use the simple text classification model described in the course with ```tf.keras.Sequential``` API :
1. An embedding layer : if you use zero padding, you can set ```mask_zero=True``` and use the ```weights``` matrix for initialization.
2. A dense layer without particular activation function for linear projection of the previous embedding vectors.
3. A global average pooling (1D).
4. A final dense layer for linear projection in a $d$-dimensional space for sentiment prediction, with $d$ the number of possible sentiments/classes.


In [ ]:
##### FILL THE BLANKS



# Improve your sentiment model coupling reviewer embeddings and word embeddigns

<img src="classif.png"></img>

#### Reviewer classification model 
Create a second sentiment model that will solely use the reviewer embeddings to predict the restaurant rating

#### Data preparation

In [ ]:
from gensim.models import Word2Vec
n2v = Word2Vec.load("node2vec.pkl")

In [ ]:
labeled_documents = df_cap[['review_content', 'rest_rating', 'reviewer_pseudo']].head(DATASET_SIZE).dropna().reset_index(drop=True)
labeled_documents.shape

1. Create a second sentiment model that will solely use the reviewer embeddings to predict the sentiment
2. Merge the first and two models to complete the above architecture. 


You will be using the functional API of keras. 

#### Proposed architecture 
**Review NLP model**  
1. Input Shape : the input text sequence of max size N (including padding)
2. Embedding Layers: mapping the embedding matrix with your input sequence
3. Dense layer: linear projection of the previous embedding
3. Global average Pooling 1D: 
4. Flatten: just flattening the representation into a vector, this is the review embedding ! 

**Reviewer (meta features) model**
5. Input Shape : the input reviewer embedding of size N' (dimension you choose for node2vec)


**Merging the two models**
6. Concatenate layer merging the two models input. 
7. Global average Pooling 1D: 
8. Flatten: Final representation before classification
9. Dense layer with softmax activation of size corresponding to the number of class


In [ ]:
sentiment_model2 = tf.keras.Sequential([
    tf.keras.layers.Dense(400, input_shape=(64,)),
    tf.keras.layers.Dense(5)
])

sentiment_model2.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                        optimizer=tf.keras.optimizers.Adam(1e-4),
                        metrics=['accuracy'])

In [ ]:
history = sentiment_model2.fit(
    X2_train, y2_train, epochs=10, validation_data=(X2_test, y2_test))

In [ ]:
sentiment_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(token_encoder.vocab_size, 300, mask_zero=True, weights=[weights], trainable=True),
    tf.keras.layers.Dense(400),
    tf.keras.layers.GlobalAveragePooling1D(data_format='channels_last'),
    tf.keras.layers.Dense(400, input_shape=(64,)),
    tf.keras.layers.Dense(5)
])

sentiment_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                        optimizer=tf.keras.optimizers.Adam(1e-4),
                        metrics=['accuracy'])
sentiment_model.summary()

In [ ]:
import tensorflow as tf
n_classes = 5

input_1 = tf.keras.layers.Input(shape=(743,))
emb = tf.keras.layers.Embedding(token_encoder.vocab_size, 300, mask_zero=True, weights=[weights], trainable=True)(input_1)
dense = tf.keras.layers.Dense(400)(emb)
pool = tf.keras.layers.GlobalAveragePooling1D(data_format='channels_last')(dense)
flatten = tf.keras.layers.Flatten()(pool)
tf.keras.layers.Dense(5)

input_2 = tf.keras.layers.Input(shape=(64,))


# Concatenate
concat = tf.keras.layers.Concatenate()([flatten, input_2])

dense_2 = tf.keras.layers.Dense(400)(concat)

# output layer
output = tf.keras.layers.Dense(units=n_classes,
                               activation=tf.keras.activations.softmax)(dense_2)
    
full_model = tf.keras.Model(inputs=[input_1, input_2], outputs=[output])

full_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                        optimizer=tf.keras.optimizers.Adam(1e-4),
                        metrics=['accuracy'])
print(full_model.summary())

In [ ]:
#X_train.shape, X2_train.shape, y_train.shape, y2_test.shape

In [ ]:
history = full_model.fit([X_train, X2_train], y_train, epochs=10, validation_data=([X_test, X2_test], y_test))